In [2]:
import pandas as pd
import numpy as np
import re
from collections import Counter

from konlpy.tag import Okt
from gensim.models import Word2Vec

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_ka = pd.read_csv('./src/comments_kakao_300000.csv')
df_sam = pd.read_csv('./src/comments_samsung_200000.csv')

In [3]:
df = df_ka.append(df_sam)
df = df.reset_index(drop=True)
print('데이터 크기: ',df.shape)
df.head()

데이터 크기:  (500000, 5)


,날짜,댓글,조회수,좋아요,싫어요
0,2022-05-03,개미들 조련하는법,26,1,0
1,2022-05-03,6만원,38,1,0
2,2022-05-03,카카오는 끝물입니다 사실상,72,2,2
3,2022-05-03,안티 살발하다,34,2,0
4,2022-05-03,이런 호로개미지옥탕을 만드신,58,3,1


In [4]:
df = df[df['댓글'] != 'error']
df = df.dropna()
print('데이터 크기: ',df.shape)
df.head()

데이터 크기:  (499984, 5)


,날짜,댓글,조회수,좋아요,싫어요
0,2022-05-03,개미들 조련하는법,26,1,0
1,2022-05-03,6만원,38,1,0
2,2022-05-03,카카오는 끝물입니다 사실상,72,2,2
3,2022-05-03,안티 살발하다,34,2,0
4,2022-05-03,이런 호로개미지옥탕을 만드신,58,3,1


In [5]:
df['정제된 댓글'] = df['댓글'].str.replace('\[삭제된 게시물의 답글\]',' ')
df['정제된 댓글'] = df['정제된 댓글'].str.replace('답글:',' ')
df['정제된 댓글'] = df['정제된 댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip()
df = df[df['정제된 댓글'] != '']
df = df.reset_index(drop=True)
df = df[['댓글','정제된 댓글']]
print('데이터 크기: ',df.shape)
df.head()

데이터 크기:  (493864, 2)


,댓글,정제된 댓글
0,개미들 조련하는법,개미들 조련하는법
1,6만원,만원
2,카카오는 끝물입니다 사실상,카카오는 끝물입니다 사실상
3,안티 살발하다,안티 살발하다
4,이런 호로개미지옥탕을 만드신,이런 호로개미지옥탕을 만드신


In [7]:
okt = Okt()
tag_list = ['Noun','Verb','Adjective','VerbPrefix']
tokenized_data = []

for i in range(df.shape[0]):
    tokenized_sentence = okt.pos(df['정제된 댓글'][i], stem=True) # 토큰화
    tag_checked_sentence = []
    for j in tokenized_sentence:
        x,y = j
        if y in tag_list:
            tag_checked_sentence.append(x)
    tokenized_data.append(tag_checked_sentence)
    print(f'\r{i+1}개 형태소분석 완료.',end='')
df['토큰화 댓글'] = tokenized_data
# df = df[df['토큰화 댓글'].str.len() > 1]
df = df.reset_index(drop=True)
df = df[df['토큰화 댓글'] != '[]']
print('\n데이터 크기: ',df.shape)
df.head()

493864개 형태소분석 완료.
데이터 크기:  (493864, 3)


,댓글,정제된 댓글,토큰화 댓글
0,개미들 조련하는법,개미들 조련하는법,"[개미, 조련, 하다, 법]"
1,6만원,만원,[만원]
2,카카오는 끝물입니다 사실상,카카오는 끝물입니다 사실상,"[카카오, 끝물, 이다, 사실]"
3,안티 살발하다,안티 살발하다,"[안티, 살발]"
4,이런 호로개미지옥탕을 만드신,이런 호로개미지옥탕을 만드신,"[이렇다, 호로, 개미, 지옥탕, 만들다]"


In [43]:
df = df.drop_duplicates('토큰화 댓글')
df = df.reset_index(drop=True)
print('\n데이터 크기: ',df.shape)
df.head()


데이터 크기:  (362544, 3)


,댓글,정제된 댓글,토큰화 댓글
0,개미들 조련하는법,개미들 조련하는법,"['개미', '조련', '하다', '법']"
1,6만원,만원,['만원']
2,카카오는 끝물입니다 사실상,카카오는 끝물입니다 사실상,"['카카오', '끝물', '이다', '사실']"
3,안티 살발하다,안티 살발하다,"['안티', '살발']"
4,이런 호로개미지옥탕을 만드신,이런 호로개미지옥탕을 만드신,"['이렇다', '호로', '개미', '지옥탕', '만들다']"


In [44]:
# df.to_csv('./src/tokenized_comment.csv', index=False)

In [30]:
df = pd.read_csv('./src/tokenized_comment.csv')
print('데이터 크기: ',df.shape)
df.head()

데이터 크기:  (362544, 3)


,댓글,정제된 댓글,토큰화 댓글
0,개미들 조련하는법,개미들 조련하는법,"['개미', '조련', '하다', '법']"
1,6만원,만원,['만원']
2,카카오는 끝물입니다 사실상,카카오는 끝물입니다 사실상,"['카카오', '끝물', '이다', '사실']"
3,안티 살발하다,안티 살발하다,"['안티', '살발']"
4,이런 호로개미지옥탕을 만드신,이런 호로개미지옥탕을 만드신,"['이렇다', '호로', '개미', '지옥탕', '만들다']"


In [31]:
df['공포탐욕'] = 0.5

In [32]:
fg_list = df['공포탐욕'].to_list()
for i in range(df.shape[0]):
    comment = eval(df['토큰화 댓글'][i])
    if '좋다' in comment:
        fg_list[i] += 0.5
        fg_list[i] = int(fg_list[i])
    if '망하다' in comment:
        fg_list[i] -= 0.5
        fg_list[i] = int(fg_list[i])
    if fg_list[i] == 0.5:
        fg_list[i] = 2
df['공포탐욕'] = fg_list

In [33]:
df

,댓글,정제된 댓글,토큰화 댓글,공포탐욕
0,개미들 조련하는법,개미들 조련하는법,"['개미', '조련', '하다', '법']",2
1,6만원,만원,['만원'],2
2,카카오는 끝물입니다 사실상,카카오는 끝물입니다 사실상,"['카카오', '끝물', '이다', '사실']",2
3,안티 살발하다,안티 살발하다,"['안티', '살발']",2
4,이런 호로개미지옥탕을 만드신,이런 호로개미지옥탕을 만드신,"['이렇다', '호로', '개미', '지옥탕', '만들다']",2
...,...,...,...,...
362539,양성전자가자!! 10만가자!!,양성전자가자 만가자,"['자가', '만가']",2
362540,답글:[삭제된 게시물의 답글]쳐맞고 있는건 석...,쳐맞고 있는건 석,"['치다', '맞다', '있다', '석']",2
362541,규모 4.9 지진,규모 지진,"['규모', '지진']",2
362542,지진났어도 뭉가리는호주에서 신나게ㅡ,지진났어도 뭉가리는호주에서 신나게,"['지진', '나다', '뭉', '가리다', '호주', '신나다']",2


In [34]:
df[df['공포탐욕']==0].append(df[df['공포탐욕']==1])

,댓글,정제된 댓글,토큰화 댓글,공포탐욕
122,제발 카카오 망해라,제발 카카오 망해라,"['제발', '카카오', '망하다']",0
165,쓰레기 구멍가게 망해라,쓰레기 구멍가게 망해라,"['쓰레기', '구멍', '가게', '망하다']",0
235,평단 91800인데 인생 망함?,평단 인데 인생 망함,"['평단', '인생', '망하다']",0
242,아직 안망했네 개잡주,아직 안망했네 개잡주,"['안', '망하다', '개', '잡다']",0
376,완전 폭삭 망했다!!!,완전 폭삭 망했다,"['완전', '폭삭', '망하다']",0
...,...,...,...,...
362200,말씀감사 실천에 좋은 약속 있다,말씀감사 실천에 좋은 약속 있다,"['말씀', '감사', '실천', '좋다', '약속', '있다']",1
362259,중국이 후회할만큼 백만프로 가상화폐 좋아...,중국이 후회할만큼 백만프로 가상화폐 좋아,"['중국', '후회', '하다', '백만', '프로', '가상', '화폐', '좋다']",1
362295,나도그래쓰면 좋?네 으음,나도그래쓰면 좋 네 으음,"['나다', '그렇다', '쓰다', '좋다', '네']",1
362420,김혜수가 선전하는 발란보다 더 좋은 빽,김혜수가 선전하는 발란보다 더 좋은 빽,"['김혜수', '선전', '하다', '발란', '더', '좋다', '빽']",1


In [35]:
df[df['공포탐욕']==0]

,댓글,정제된 댓글,토큰화 댓글,공포탐욕
122,제발 카카오 망해라,제발 카카오 망해라,"['제발', '카카오', '망하다']",0
165,쓰레기 구멍가게 망해라,쓰레기 구멍가게 망해라,"['쓰레기', '구멍', '가게', '망하다']",0
235,평단 91800인데 인생 망함?,평단 인데 인생 망함,"['평단', '인생', '망하다']",0
242,아직 안망했네 개잡주,아직 안망했네 개잡주,"['안', '망하다', '개', '잡다']",0
376,완전 폭삭 망했다!!!,완전 폭삭 망했다,"['완전', '폭삭', '망하다']",0
...,...,...,...,...
359943,민주당 재집권하면 삼전망한다,민주당 재집권하면 삼전망한다,"['민주당', '재', '집권', '삼전', '망하다']",0
360042,중국이 잘 되어도 망해도 한국은 불행,중국이 잘 되어도 망해도 한국은 불행,"['중국', '자다', '되어다', '망하다', '한국', '불행']",0
360092,여러분 집안은 망한거야.,여러분 집안은 망한거야,"['여러분', '집안', '망하다']",0
360175,신은 망하길 원해ㅠ,신은 망하길 원해,"['신다', '망하다', '원하다']",0


In [37]:
df[df['공포탐욕']==1]

,댓글,정제된 댓글,토큰화 댓글,공포탐욕
219,매수하기 딱 좋은 날~~^,매수하기 딱 좋은 날,"['매수', '하다', '좋다', '날']",1
281,플랫폼 기업들 코로나로 좋았지....,플랫폼 기업들 코로나로 좋았지,"['플랫폼', '기업', '코로나', '좋다']",1
306,어차피 낼 하고 모레는 지수 좋을꺼고,어차피 낼 하고 모레는 지수 좋을꺼고,"['어차피', '낼', '하다', '모레', '지수', '좋다']",1
462,좋네 ㅎ,좋네,['좋다'],1
504,조져따리,조져따리,"['좋다', '따리']",1
...,...,...,...,...
362200,말씀감사 실천에 좋은 약속 있다,말씀감사 실천에 좋은 약속 있다,"['말씀', '감사', '실천', '좋다', '약속', '있다']",1
362259,중국이 후회할만큼 백만프로 가상화폐 좋아...,중국이 후회할만큼 백만프로 가상화폐 좋아,"['중국', '후회', '하다', '백만', '프로', '가상', '화폐', '좋다']",1
362295,나도그래쓰면 좋?네 으음,나도그래쓰면 좋 네 으음,"['나다', '그렇다', '쓰다', '좋다', '네']",1
362420,김혜수가 선전하는 발란보다 더 좋은 빽,김혜수가 선전하는 발란보다 더 좋은 빽,"['김혜수', '선전', '하다', '발란', '더', '좋다', '빽']",1
